# Global Analysis of Renewable and Non-Renewable Energy Distribution
### Author: Mario Gonzalez

## Objective
This Jupyter Notebook uses various formatted databases to extract
meaningful data and visualize global power distribution. The analysis
focuses on the worldwide distribution of installed capacity for both
renewable and non-renewable energy sources, as well as their evolution
over time. It also takes into account subsidies for renewable and
non-renewable technologies by country.

### Disclaimer
This script is provided as-is, without any warranty or guarantee. Use it
at your own risk.

# Data Cleaning and Processing

This section involves the cleaning and processing of the following databases,  
which will be used in the subsequent analysis.

## Data Sources

- **Global Power Plant Database**  
  https://datasets.wri.org/datasets/global-power-plant-database

- **Fossil Fuel Subsidies**  
  https://www.iea.org/topics/fossil-fuel-subsidies

- **Government Energy Spending Tracker: Policy Database**  
  https://www.iea.org/data-and-statistics/data-tools/government-energy-spending-tracker-policy-database

- **World Bank API**  
  https://documents.worldbank.org/en/publication/documents-reports/api

In [18]:
# Imports
import pandas as pd
import re
import requests
import csv

## Global Power Plant Database

The database is already clean and well-formatted in a `.csv` file.  
Using `pandas`, it is easy to load the data into a DataFrame. Since the  
database is quite large and much of the analysis relies on it, a detailed  
description is provided during the execution of the notebook.

In [ ]:
# Column data type since the db is quite big and infering type with pandas is
# quite memory demanding
dtype_dict = {
    "country": str,
    "country_long": str,
    "name": str,
    "gppd_idnr": str,
    "capacity_mw": float,
    "latitude": float,
    "longitude": float,
    "primary_fuel": str,
    "other_fuel1": str,
    "other_fuel2": str,
    "other_fuel3": str,
    "commissioning_year": float,
    "owner": str,
    "source": str,
    "url": str,
    "geolocation_source": str,
    "wepp_id": str,
    "year_of_capacity_data": float,
    "generation_gwh_2013": float,
    "generation_gwh_2014": float,
    "generation_gwh_2015": float,
    "generation_gwh_2016": float,
    "generation_gwh_2017": float,
    "generation_gwh_2018": float,
    "generation_gwh_2019": float,
    "generation_data_source": str,
    "estimated_generation_gwh_2013": float,
    "estimated_generation_gwh_2014": float,
    "estimated_generation_gwh_2015": float,
    "estimated_generation_gwh_2016": float,
    "estimated_generation_gwh_2017": float,
    "estimated_generation_note_2013": str,
    "estimated_generation_note_2014": str,
    "estimated_generation_note_2015": str,
    "estimated_generation_note_2016": str,
    "estimated_generation_note_2017": str,
}

# DtypeWarning: Columns (10) have mixed types (other_fuel3) -> it should be str
df_global_power_plant = pd.read_csv(
    "original-databases\\GlobalPowerPlantDB.csv",
    dtype=dtype_dict,
)

C:\Users\Mario\AppData\Local\Temp\ipykernel_16252\3986076487.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_global_power_plant = pd.read_csv("original-databases\GlobalPowerPlantDB.csv")
C:\Users\Mario\AppData\Local\Temp\ipykernel_16252\3986076487.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_global_power_plant = pd.read_csv(


## Fossil Fuel Subsidies Database

The database is provided in `.xlsx` format.  
The first step is to list all the available sheets and extract the data  
from each one individually.


In [20]:
# Load the Excel file
file_path = "original-databases/Subsidies 2010-2023.xlsx"
xls = pd.ExcelFile(file_path)

# List all sheet names
display(xls.sheet_names)

# Expected result:
# ['Subsidies by country', 'Indicators by country', 'Transport Oil Subsidies']

['Subsidies by country', 'Indicators by country', 'Transport Oil Subsidies']

### Subsidies by Country Sheet

This first sheet contains two tables, covering data from 2010 to 2023.  
The first table shows the total subsidies for all countries, in million USD.  
The second table presents subsidy values for each individual country.


In [21]:
# Load the "Subsidies by country" sheet into a DataFrame
df_subsidies_by_country = pd.read_excel(file_path, sheet_name="Subsidies by country", header=None)

# Extract the first table
total_subsidies_table_raw = df_subsidies_by_country.iloc[4:10].copy()
# Use the first row of this slice as the header
total_subsidies_table_raw.columns = total_subsidies_table_raw.iloc[0]
# Exclude the header row from the data and reset the index
total_subsidies_table = total_subsidies_table_raw.iloc[1:].reset_index(drop=True)

# Extract the second table
subsidies_by_country_raw = df_subsidies_by_country.iloc[11:252].copy()
# Use the first row of the second table as the header
subsidies_by_country_raw.columns = subsidies_by_country_raw.iloc[0]
# Drop the header row and reset the index
subsidies_by_country_table = subsidies_by_country_raw.iloc[1:].reset_index(drop=True)

# Save both tables into CSV to check the dataframes were correctly loaded
total_subsidies_table.to_csv('cleaned-databases/total-subsidies-table.csv', index=False)
subsidies_by_country_table.to_csv('cleaned-databases/subsidies-by-country.csv', index=False)

### Indicators by Country Sheet

This sheet is simpler and contains a single table,  
with one country per row.

In [22]:
# Load the "Subsidies by country" sheet into a DataFrame
df_indicators_by_country = pd.read_excel(file_path, sheet_name="Indicators by country", header=None)

# Use the first row of the actual table as the header
indicators_by_country_raw = df_indicators_by_country.iloc[3:44].copy()
indicators_by_country_raw.columns = indicators_by_country_raw.iloc[0]

# Exclude the header row from the data and reset the index
indicators_by_country = indicators_by_country_raw.iloc[1:].reset_index(drop=True)

# Save table into CSV to check the dataframe was correctly loaded
indicators_by_country.to_csv('cleaned-databases/indicators-by-country.csv', index=False)

### Transport Oil Subsidies

This sheet contains a single table showing transport oil subsidies,  
reported in real 2023 million USD.

In [23]:
# Load sheet into dataframe
df_transport_oil_subsidies = pd.read_excel(file_path, sheet_name="Transport Oil Subsidies", header=None)


# Use the first row of the table as the header
transport_oil_subsidies_raw = df_transport_oil_subsidies.iloc[4:54].copy()
transport_oil_subsidies_raw.columns = transport_oil_subsidies_raw.iloc[0]
# Exclude the header row from the data and reset the index
transport_oil_subsidies = transport_oil_subsidies_raw.iloc[1:].reset_index(drop=True)

# Save table into CSV to check the dataframe was correctly loaded
transport_oil_subsidies.to_csv('cleaned-databases/transport-oil-subsidies.csv', index=False)

### Subsidies for Clean Energy

There is no straightforward way to download the raw data provided by the IEA.  
However, they offer interactive tables and graphs. It is possible to copy the  
table data into a `.txt` file, though the formatting is inconsistent.

Regular expressions were used to clean the data, as tabs, carriage returns (CR),  
and blank lines were mixed as separators.

The total is also displayed on the website:  
**Total commitment:** 2,957.26 billion USD  
**Largest spending area:** Energy affordability

The strategy used involves identifying the end of each line by detecting digits,  
since the last column (budget) is the only numeric one.  
However, it's important to account for rows that contain "`< 1`" as the budget value.

Function docstrings provide more detailed explanations of the cleaning logic.

In [24]:

# Assuming the budget line should contain only digits, spacing characters or "< 1".
def is_budget_line(line):
    """
    Return True if the line appears to be a budget commitment.
    The function also accepts an optional leading '<' or '>' before
    a number. It also strips non-breaking spaces.
    """
    # Remove standard whitespace and common non-breaking spaces.
    clean = line.strip().replace("\u202f", "").replace("\xa0", "")
    #   ^[<>]?\s* => Allows an optional < or > followed by any spaces.
    #   [\d\s]+   => Must have at least one digit (allowing spaces as thousands separators).
    #   (?:[.,]\d+)?$ => Optionally followed by a decimal separator and digits.
    pattern = r'^[<>]?\s*[\d\s]+(?:[.,]\d+)?$'
    return bool(re.fullmatch(pattern, clean))

def parse_record(record_lines):
    """
    Given a list of lines for one record, extract:
    Country/region, Policy, Measures, Start year, Status, Budget commitment.
    
    The assumptions:
      - First line: country and policy separated by a tab.
      - Last line: budget commitment.
      - Among the remaining lines, the highest-indexed line that contains a tab
        is assumed to be the "year line" with tokens: [optional measure part, start year, status].
      - All lines (if any) between the first and the year line are additional measures.
    """
    if len(record_lines) < 2:
        return None  # not enough info
    
    # First line: country and policy.
    first_line_tokens = record_lines[0].split('\t')
    country = first_line_tokens[0].strip()
    policy = first_line_tokens[1].strip() if len(first_line_tokens) > 1 else ''
    
    # Last line is the budget.
    budget = record_lines[-1].strip()
    # Remove blank space if necessary (e.g. 30 000)
    budget = re.sub(r"[\u202f\u00a0\s]", "", record_lines[-1])

    # Identify the "year line": search from the end (excluding the budget)
    year_line_idx = None
    for i in range(len(record_lines) - 2, 0, -1):  # from second-to-last to second line
        if '\t' in record_lines[i]:
            year_line_idx = i
            break
    if year_line_idx is None:
        # Fallback: if not found, then assume the second line is the year line.
        year_line_idx = 1
    
    # Process the year line.
    year_line_tokens = [token.strip() for token in record_lines[year_line_idx].split('\t')]
    # The first token from the year line might be a continuation of measures.
    additional_measure = year_line_tokens[0] if year_line_tokens[0] else ''
    start_year = year_line_tokens[1] if len(year_line_tokens) > 1 else ''
    status = year_line_tokens[2] if len(year_line_tokens) > 2 else ''
    
    # Other lines between the first line and the identified year line are also measures.
    measure_parts = []
    if year_line_idx > 1:
        for line in record_lines[1:year_line_idx]:
            measure_parts.append(line.strip())
    # Add the first token of the year line as well.
    if additional_measure:
        measure_parts.append(additional_measure)
    measures = " ".join(measure_parts).strip()
    
    return [country, policy, measures, start_year, status, budget]

# Read the original file
with open('original-databases\Government Energy Spending Tracker Policy Database.txt', "r", encoding="utf-8") as f:
    lines = f.readlines()

# Remove header line and strip whitespace.
header_line = lines[0].strip()
headers = [h.strip() for h in header_line.split(',')]
data_lines = [line.rstrip() for line in lines[1:] if line.rstrip() != ""]

# Group lines into records based on detection of budget lines.
records = []
current_record = []

for line in data_lines:
    # Append the line to the current record.
    current_record.append(line)
    # If this line qualifies as a budget line, assume the record is complete.
    if is_budget_line(line):
        records.append(current_record)
        current_record = []

# Parse each record into a list of fields.
parsed_rows = []
for rec in records:
    parsed = parse_record(rec)
    if parsed:
        parsed_rows.append(parsed)
    else:
        print("Warning: Failed parsing record:", rec)

# Create a DataFrame and save the result to a CSV.
df_renewable_subsidies = pd.DataFrame(parsed_rows, columns=headers)
df_renewable_subsidies.to_csv("cleaned-databases/renewable-subsidies.csv", index=False)

### World Bank API

The World Bank provides access to its API without requiring authentication.  
Using this, the population data for all countries was queried starting from 2013  
to perform per capita calculations. The API offers both XML and JSON responses,  
and the latter was chosen for this analysis, following the World Bank API Best
Practices Guideline.

In [25]:
# World Bank API URL for total population from 2013 to 2019
url = "https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?date=2013:2019&per_page=6000&format=json"

try:
    response = requests.get(url)
    response.raise_for_status()

    data = response.json()
    population_data = data[1]  # List of population records

    # Write a CSV file with the fetched response
    with open("cleaned-databases/world_population_2013_2019.csv", mode="w", newline='', encoding="utf-8") as file:
        writer = csv.writer(file)

        # CSV header
        writer.writerow(["Country", "Country ISO3 Code", "Year", "Population"])

        # Write only rows where population value exists (Smaller territories do not have a value)
        for item in population_data:
            if item['value'] is not None:
                writer.writerow([
                    item['country']['value'],
                    item['country']['id'],
                    item['date'],
                    item['value']
                ])

except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"An error occurred: {err}")


# Visualization

In [26]:
# Power plant distribution

In [27]:
# Installed power capacity distribution

In [28]:
# Renewable and non-renewable energy distribution

In [29]:
# Renewable energy subsidies per country

In [30]:
# Renewable energy subsidies per country and per capita

In [31]:
# Fossil fuel subsidies per country

In [32]:
# Fossil fuel subsidies per country and per capita

In [33]:
# Renewable and non-renewable energy evolution

In [34]:
# Renewable and non-renewable energy per continent